# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [1]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import json
import re

import time
from typing import Dict, List

from bs4 import BeautifulSoup

import requests

import schedule

from tqdm import tqdm

# Для повторного использования базового соединения и для ускорения парсинга
s_requests = requests.Session()
# Переход на каталог выше (из каталога notebooks/)
# Для обеспечения идентичности кода в ноутбуке и в файле scraper.py
# А так же для работы тестов
%cd ../

/home/brasavar/Документы/MIPT/Python/books_scraper


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [12]:
def get_book_data(book_url: str) -> dict:
    """
    Сбор данных о книге с одной страницы парсингом.
    Parameters:  
    book_url (str): Адрес страницы для парсинга.  
    Returns:  
    dict: Собранные данные о книге.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Получаем страницу по ардесу book_url
    responce = s_requests.get(book_url, timeout=5)
    # Для корректной обработки значения цены
    responce.encoding = 'utf-8'
    # Словарь для вывода данных по результатам работы функции
    res: dict = {}
    # Убеждаемся, что наш запрос отработал корректно и вернул код состояния 200
    if responce.status_code != 200:
        raise ConnectionError(f"Can't connect to {responce.status_code}")
    # Разбираем парсером bs4 полученный в запросе текст
    soup: BeautifulSoup = BeautifulSoup(responce.text, 'html.parser')
    # Находим название книги по ближайшему уникальному тегу и его атрибуту
    tag_name = soup.find('div', attrs={'class': 'col-sm-6 product_main', })
    try:
        # Добавляем название книги в словарь вывода с ключем Name
        res['Title'] = tag_name.h1.text
    except AttributeError:
        res['Title'] = "Нет описания имени книги на сайте"
    # Находим описание книги по соседнему уникальному тегу и его атрибуту
    sibling = soup.find('div', attrs={'id': 'product_description', })
    try:
        # Добавляем описание книги в словарь вывода с ключем Description
        res['Description'] = sibling.find_next_sibling().text
    except AttributeError:
        res['Description'] = 'Нет описания на сайте'
    # Находим рейтинг книги по тегу 'p' и его атрибуту 'class'
    rating = soup.find('p', attrs={'class': 'star-rating',})
    try:
        # Добавляем рейтинг книги в словарь вывода с ключем Rating
        res['Rating'] = rating.attrs['class'][1]
    except AttributeError:
        res['Rating'] = 'Нет рейтинга книги на сайте'
    # Находим по тегу и его уникальному атрибуту таблицу с остальными
    # характеристиками книги. Через children получаем итератор по вложенным
    # тегам.
    table_inf = soup.find('table', attrs={'class': 'table table-striped'})
    if table_inf:
        product_inf = table_inf.children
        # Проходим в цикле по итератору
        for row in product_inf:
            # В тегах, кроме искомых th - название характеристики и
            # td - значение характеристики, есть еще мешающий тег tr
            # его обрабатываем try/except (обеспечение консистентности).
            try:
                # Через пары th:td (название:значение) дополняем словарь вывода
                res[row.th.text] = row.td.text
            except AttributeError:
                pass
    return res
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [13]:
# Используйте для самопроверки
book_url: str = (
    'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
)

get_book_data(book_url)

{'Title': 'A Light in the Attic',
 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone do

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [14]:
def scrape_books(is_save: bool, url: str) -> List[Dict]:
    """
    Проход в цикле по страницам каталога, с парсингом всех страниц в цикле
    с использованием функции get_book_data.
    Parameters:  
    is_save (bool): Осуществлять ли запись в файл.  
    Returns:  
    list(dict,): Собранные данные о книге.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    list_return = []
    number_of_pages = get_number_of_pages(url)
    urls = get_urls_books(url, number_of_pages)
    # Проход циклом по списку ссылок на книги, вызывая get_book_data для каждой
    for url_b in tqdm(urls):
        list_return.append(get_book_data(url_b))
    # Сохраняем собранную информацию в файл, если задан соответствующий ключ
    if is_save:
        with open('artifacts//books_data.txt', 'w', encoding='utf-8') as file:
            # Записываем данные о книгах построчно
            # В каждой новой строке словарь данных по новой книге
            for res_dict in list_return:
                json.dump(res_dict, file)
    return list_return


# Адрес первой страницы каталога книг
URL = 'http://books.toscrape.com/catalogue/page-1.html'


def get_number_of_pages(url: str) -> int:
    '''
    Получение количества страниц каталога по первой странице в каталоге.
    Parameters:
    url (str): Строка с адресом первой страницы в каталоге
    Returns:
    int: Количество страниц в каталоге
    '''
    responce = s_requests.get(url, timeout=5)
    # Проверяем, что страница доступна
    if responce.status_code != 200:
        raise ConnectionError(f"Can't connect to {responce.status_code}")
    # Проходим парсингом bs4 по полученной странице
    soup = BeautifulSoup(responce.text, 'html.parser')
    # Находим поле, описывающее количество страниц в каталоге
    current_page = soup.find('li', attrs={'class': 'current'})
    # Пытаемся извлечь текст количества страниц в каталоге, преобразуем в int
    try:
        number_of_pages = int(current_page.text.split()[-1])
    except AttributeError:
        pass
    return number_of_pages


def get_urls_books(url: str, number_of_pages: int) -> List[str]:
    '''
    Получение url всех книг в каталоге
    Parameters:
    url (str): Строка с адресом первой страницы в каталоге
    number_of_pages (int): Количество страниц в каталоге
    Returns:
    list(str): Список url всех книг в каталоге
    '''

    # Пустой список, для наполнения строками адресов книг
    urls = []
    # Проходим циклом по каждой странице каталога
    for i in range(1, number_of_pages+1):
        # Не меняющаяся часть url + номер страницы каталога
        url_p = f'{re.search('.+page-', url)[0]}{str(i)}.html'
        # Проверка на доступность страницы каталога
        resp_book = s_requests.get(url_p, timeout=5)
        if resp_book.status_code != 200:
            raise ConnectionError(f"Can't connect to {resp_book.status_code}")
        # Проходим парсингом bs4 по странице каталога
        soup_book = BeautifulSoup(resp_book.text, 'html.parser')
        pattern = r'.+catalogue/'
        # Формируем список url, используя не меняющуюся часть url каталога,
        # добавляем к нему относительный адрес книги, найденный в атрибуте
        # href=... списка тегов книг <a href=....>...</a>
        # Список ссылок формируем накоплением.
        urls += list(
            map(
                lambda s: f'{re.search(pattern, url)[0]}{s.a.attrs['href']}',
                soup_book.find('ol', class_='row').find_all('h3')
            )
        )
    return urls
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [9]:
# Проверка работоспособности функции
res = scrape_books(is_save=True, url=URL) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

100%|██████████| 1000/1000 [02:42<00:00,  6.16it/s]

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
schedule.every().day.at("19:00").do(lambda: scrape_books(
        is_save=True, url=URL
        )
    )
while True:
    schedule.run_pending()
    time.sleep(1)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

100%|██████████| 1000/1000 [02:46<00:00,  6.01it/s]


KeyboardInterrupt: 

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [2]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

============================= test session starts ==============================
platform linux -- Python 3.12.3, pytest-8.4.2, pluggy-1.6.0
rootdir: /home/brasavar/Документы/MIPT/Python/books_scraper
plugins: mock-3.15.1
collected 4 items                                                              

tests/test_scraper.py ....                                               [100%]

============================== 4 passed in 4.26s ===============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```